In [1]:
#nbi:hide_in
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel, RBF

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm
import numpy
import nbinteract as nbi
import seaborn as sns
import scipy

In [2]:
#nbi:hide_in
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from keras.layers import LeakyReLU

train_data_source = '/Users/anubhavjain/Downloads/nlp_assignment_3/train.csv'
test_data_source = '/Users/anubhavjain/Downloads/nlp_assignment_3/test.csv'

train_df = pd.read_csv(train_data_source, delimiter="\t")
test_df = pd.read_csv(test_data_source, delimiter="\t")

Using TensorFlow backend.


In [3]:
#nbi:hide_in
train_df.head()

,meta,uid,sentiment
0,meta,3,negative
1,@,O,NaN
2,AdilNisarButt,Hin,NaN
3,pakistan,Hin,NaN
4,ka,Hin,NaN


In [4]:
#nbi:hide_in
test_df.head()

,meta,uid,sentiment
0,meta,8,neutral
1,RT,Eng,NaN
2,@,O,NaN
3,UAAPconfessions,Eng,NaN
4,Love,Eng,NaN


In [5]:
#nbi:hide_in
string = ""
a = train_df.loc[:,['meta','sentiment']]
temp = ""
train_arr = []
train_arr_senti = []

for i in range(a.shape[0]):
  if(a.iloc[i,0]=="meta"):
    #print(a.iloc[i,0])
    train_arr.append(temp)
    train_arr_senti.append(str(a.iloc[i,1]))
    temp=""
  else:
    temp+=str(a.iloc[i,0])+" "

train_arr.append(temp)
train_arr=train_arr[1:]
print(train_arr[0],train_arr_senti[0])

@ AdilNisarButt pakistan ka ghra tauq he Pakistan Israel ko tasleem nahein kerta Isko Palestine kehta he - OCCUPIED PALESTINE  negative


In [6]:
#nbi:hide_in
string = ""
a = test_df.loc[:,['meta','sentiment']]
temp = ""
test_arr = []
test_arr_senti = []

for i in range(a.shape[0]):
  if(a.iloc[i,0]=="meta"):
    #print(a.iloc[i,0])
    test_arr.append(temp)
    test_arr_senti.append(str(a.iloc[i,1]))
    temp=""
  else:
    temp+=str(a.iloc[i,0])+" "

test_arr.append(temp)
test_arr=test_arr[1:]
print(test_arr[0],test_arr_senti[0])

RT @ UAAPconfessions Love looks good on Maddie !!! Ako lang ba yung sobrang masaya kasi may zolo sya ? Before with the past Z medyo lowkey s …  neutral


In [7]:
#nbi:hide_in
from nltk.tokenize import TweetTokenizer
import nltk
import string
import sys
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from nltk.corpus import wordnet
import math
import random


punc1 = string.punctuation
punc = []
for i in punc1:
    punc.append(i)

#print(punc, len(punc))
#print("Default encoding is .. ",sys.getdefaultencoding())

tknr = TweetTokenizer(strip_handles=True)
sum = 0
count_punc = 0
arr = []

heap_tok = []
heap_voc = []
c_tok = 0
c_voc = 0
s_voc = set()
arr_tweet = []

for i in train_arr:
    a = tknr.tokenize(i)
    b = []
    arr1 = []
    for j in a:
        if(re.match(r"\.[\s.]*\.",j)):
            count_punc+=1
        elif(j in punc):
            count_punc+=1
        elif(re.match(r"https:\/\/\S*|pic.twitter\S*|http|Eng|Hin|positive|neutral|negative|RT|co",j)):
            count_punc+=1
        else:
            arr.append(j)
            arr1.append(j)
            b.append(j)
            s_voc.add(j)
    arr_tweet.append(" ".join(arr1))
    c_tok+=len(b)
    c_voc= len(s_voc)
    heap_tok.append(c_tok)
    heap_voc.append(c_voc)
    sum+=len(a)

no_tokens = len(arr)
print("No. of tokens ", no_tokens)

type_arr = np.unique(arr)
no_type = len(type_arr)

print("No. of Types", no_type)
print("TTR",no_type/no_tokens)

No. of tokens  337152
No. of Types 65222
TTR 0.19344983864844342


In [8]:
len(arr_tweet)

13524

In [9]:
#nbi:hide_in
def word_map_generation(num):
    all_words = ' '.join([text for text in arr_tweet[0:num]])
    from wordcloud import WordCloud
    wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()
word_map_generation = interact(word_map_generation, num=(0,5000));

interactive(children=(IntSlider(value=2500, description='num', max=5000), Output()), _dom_classes=('widget-int…

# Sentiment Analysis on Twitter Data

Sentiment Analysis is the automated process of analyzing text data and determining the sentiment of the opinion into positive, negative or neutral. Performing Sentiment Analysis on Twitter data can help companies obtain qualitative insights to understand how people are talking about their brand.

With more than 321 million active users, sending a daily average of 500 million Tweets, Twitter has become one of the top social media platforms for news, information, and interaction with brands and influential figures around the world. Therefore, it is no surprise that companies consider this microblogging platform an essential channel for their marketing strategy and also, to provide customer service.

Twitter allows businesses to reach a broad audience and connect with customers without intermediaries. On the downside, it can be harm a brand’s reputation if negative content about the brand suddenly goes viral – you might end up with an unexpected PR crisis on your hands. This is one of the reasons why social listening ― monitoring conversation and feedback in social media ― has become a crucial process in social media marketing.

Monitoring Twitter allows companies to understand their audience, keep on top of what’s being said about their brand and their competitors, and discover new trends in the industry. However, when it comes to analyzing Twitter data, quantitative metrics like the number of mentions or retweets are not enough to get a full picture of a situation. What really counts is being able to grasp the nuance of those mentions. Are users talking positively or negatively about a product? And that’s exactly what sentiment analysis determines. It provides qualitative insights on what is being said about a topic or brand.

# What is Sentiment Analysis?

Sentiment analysis (a.k.a opinion mining) is the automated process of identifying and extracting the subjective information that underlies a text. This can be either an opinion, a judgment, or a feeling about a particular topic or subject. The most common type of sentiment analysis is called ‘polarity detection’ and consists in classifying a statement as ‘positive’, ‘negative’ or ‘neutral’.

For example, let’s take this sentence: “I don’t find the app useful: it’s really slow and constantly crashing”. A sentiment analysis model would automatically tag this as Negative.
A sub-field of Natural Language Processing (NLP), sentiment analysis has been getting a lot of attention in recent years due to its many exciting applications in a variety of fields, ranging from business to political studies.

# Why is Twitter Sentiment Analysis important?

Nearly 80% of the world’s digital data is unstructured, and data obtained from social media sources is no exception to that. Since the information is not organized in any predefined way, it’s difficult to sort and analyze. Fortunately, thanks to the developments in Machine Learning and NLP, it is now possible to create models that learn from examples and can be used to process and organize text data.

Twitter sentiment analysis systems allow you to sort large sets of tweets and detect the polarity of each statement automatically. And the best part, it’s fast and simple, saving teams valuable hours and allowing them to focus on tasks where they can make a bigger impact.

# Advantages of Twitter Sentiment Analysis

These are some of the main advantages of Twitter sentiment analysis:
> **Scalability:** let’s say you need to analyze hundreds of tweets mentioning a brand. While you could do that manually, it would take hours and hours of manual processing and would end up being inconsistent and impossible to scale. By performing Twitter sentiment analysis you can automate this task and obtain cost-effective results in a very short time.

> **Real-Time Analysis:** Twitter sentiment analysis is critical to notice sudden shifts in customer moods, detect if critics and complaints are increasing and take action before the problem escalates. You can be monitoring your brand in real-time and get valuable insights that allow you to make changes or improvements when needed.

> **Consistent Criteria:** analyzing sentiment in a text is a subjective task. When done manually, the same tweet may be perceived differently by two members of the same team, and the results will probably be biased. By training a machine learning model to perform sentiment analysis on Twitter, you can set the parameters to analyze all your data and obtain more consistent and accurate results.

# How to Use Sentiment Analysis with Twitter Data?

We can identify four main steps in this process:
- **Data Preparing: Preprocessing and Cleaning Tweets**
<br>
Once you’ve captured the tweets you need for your sentiment analysis, it’s time to prepare the data. As we mentioned earlier, social media data is unstructured. That means it’s raw, noisy and needs to be cleaned before we can start working on our sentiment analysis model. This is an important step because the quality of the data will lead to more reliable results.
Preprocessing a Twitter dataset involves a series of tasks like removing all types of irrelevant information like emojis, special characters, and extra blank spaces. It can also involve making format improvements, delete duplicate tweets, or tweets that are shorter than three characters.
- **Creating a Twitter Sentiment Analysis Model**
<br>
Since the tweets dataset provided contains both Hindi and English mixed words, it was not possible to create embeddings at word level. So, the embeddings at character level were created. The total unique tokens were 900 after tokenization was done. Since the maximum number of characters of a tweet are 140, all tweets were post padded to make the maximum size of a tweet to length 150. The model has the following architecture which consists of Convolution1D, Maxpooling1D and Dense layers:
- **Visualization of the results**
<br>
<hr>